In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

import undetected_chromedriver as uc 
import cfscrape
import time

In [105]:
# Make scraper that bypasses cloudflare
scraper = cfscrape.create_scraper()

# Create undetected chrome instance
options = uc.ChromeOptions()
options.headless = False
driver = uc.Chrome(options=options)

url = 'https://www.edeka24.de/#'
driver.get(url)




C:\Users\mjjyo\AppData\Local\Temp\ipykernel_21384\1700831825.py:6: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False


In [4]:
cookie = driver.find_element(By.XPATH, "//ul[contains(@class, 'nav-list-main') and contains(@class, 'is-level-2')]")
li_elements = cookie.find_elements(By.XPATH, ".//li[contains(@class, 'nav-item-main is-level-3')]")

links = []

for item in li_elements:
    link_holder = item.find_element(By.XPATH, ".//a")
    href = link_holder.get_attribute("href")
    links.append(href)


In [5]:
links

['https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Kaffee-ganze-Bohnen/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Kaffee-gemahlen-Instant/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Kaffeepads/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Kaffeekapseln/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Cappuccino-Kakao/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Filter/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Milch-Sahne-Alternativen/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Tee/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Getraenke/Energydrinks/?force_sid=0f43agfptrr9ccns23q0jpbt6p',
 'https://www.edeka24.de/Lebensmittel/Getraenke/Saefte/?force_sid

In [99]:
data = {'category': [], 'sub_category': [], 'product_item': [], 'product_name': [], 'product_price': [], 'product_per_kg/lr': [], 'shipping_cost':[], 'shipping_duration': [], 'date_time': [], 'prod_description': [], 'product_url': [], 'prod_image': []}

df = pd.DataFrame(data)
df

,category,sub_category,product_item,product_name,product_price,product_per_kg/lr,shipping_cost,shipping_duration,date_time,prod_description,product_url,prod_image


In [100]:
import datetime

In [15]:
def get_product_info_links(webdriver, original_page):
    webdriver.get(original_page)

    # This keeps scrolling down and pressing more products until there is no option to anymore
    going = True
    while going is True:
        try:

            # Displaying all food items

            wait = WebDriverWait(driver, 10)
            element = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(text(), 'Weitere Produkte anzeigen')]")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            time.sleep(3.235)
            driver.execute_script("window.scrollBy(0, -325)")



            time.sleep(2.2643)

            # click on the element
            element.click()
            time.sleep(4.5612)


        except:
            going =  False
    
    
    
    
    
    all_products_ul = webdriver.find_element(By.ID, 'jqList')
    all_products = webdriver.find_elements(By.XPATH, ".//li[contains(@class, 'js-init')]")
    
    all_products_links = []
    
    for item in all_products:
        product_link_holder = item.find_element(By.XPATH, ".//a")
        product_href = product_link_holder.get_attribute("href")
        all_products_links.append(product_href)
    return all_products_links

In [19]:
every_product = []
for link in links:
    all_products = get_product_info_links(driver, link)
    every_product = every_product + all_products

In [102]:
def get_product_details(webdriver, link):
    webdriver.get(link)
    wait = WebDriverWait(driver, 10)

    try:
        top_classifiers = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "breadcrumb")))

        top_classifiers = top_classifiers.text
        top_classifiers = top_classifiers.split('\n')[-3:]

        category = top_classifiers[0]
        sub_category = top_classifiers[1]
        product_item = top_classifiers[2]

        detail_box = webdriver.find_element(By.XPATH, "//div[contains(@class, 'detail-description')]")
        product_name = detail_box.find_element(By.XPATH, ".//h1").text

        product_price_details = detail_box.find_element(By.XPATH, ".//div[contains(@class, 'price')]").text
        product_price_details = product_price_details.split('\n')

        product_price = product_price_details[0]
        product_price_per_weight = product_price_details[1]
        product_shipping_cost = product_price_details[2]
        product_shipping_time = product_price_details[3]

        time_scraped = str(datetime.datetime.now())

        product_description = webdriver.find_element(By.XPATH, ".//div[contains(@class, 'row') and contains(@class, 'article-description-wrap') and contains(@class, 'open')]").text
        product_description = ' '.join(product_description.split('\n'))

        prod_image = webdriver.find_element(By.XPATH, "//div[contains(@class, 'detail-image')]")
        prod_image_select = prod_image.find_element(By.XPATH, ".//img")
        image_url = prod_image_select.get_attribute('src')

        return [category, sub_category, product_item, product_name, product_price, product_price_per_weight, product_shipping_cost, product_shipping_time, time_scraped, product_description, link, image_url]
    except:
        return None

In [106]:
for i in range(len(every_product)):
    product = every_product[i]
    product_details = get_product_details(driver, product)
    
    if product_details is not None:
        df.loc[i] = product_details
    else:
        continue

In [107]:
df.to_csv(f"C:\\Users\\mjjyo\\OneDrive\\Desktop\\omdena\\edeka_products.csv")

In [109]:
driver.quit()